# Bert 임베딩 생성하기

Hugging Face의 오픈 소스 라이브러리인 `transformers` 설치

In [1]:
!pip uninstall transformers  # transformers-3.5.1 not working in colab 2022.01.19

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.7 MB/s 
     |████████████████████████████████| 895 kB 64.7 MB/s 
     |████████████████████████████████| 3.3 MB 69.8 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


모듈 임포트

In [2]:
from transformers import BertModel, BertTokenizer
import torch

## BERT의 최상위 인코더 계층에서만 임베딩 추출

사전 학습된 BERT 모델 사용
- 사용 가능한 모든 사전 학습된 BERT 모델 확인 - https://huggingface.co/models?sort=downloads&search=bert

`bert-base-uncased` 사용
- 12개의 인코더가 있는 BERT 기반 모델이며 모두 소문자로 변환한 uncased 토큰으로 학습 
- BERT-base를 사용하고 있으므로 표현 벡터(임베딩) 크기는 768
    - 표현 벡터의 크기 = 은닉 벡터의 길이

bert-base-uncased 모델 다운로드 및 로드

In [3]:
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert-base-uncased 모델을 사전학습할 때 사용한 tokenizer 다운로드 및 로드

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

### 입력 전처리하기

문장 정의

In [5]:
sentence = 'I love Paris'

tokenizer를 이용해 문장 토큰화

In [6]:
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'paris']


토큰 리스트 시작 부분에 `[CLS]` 토큰을 추가하고 끝에 `[SEP]` 토큰을 추가

In [7]:
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


토큰 리스트의 길이를 7로 유지해야 한다고 가정하면, 모자란 개수(2개)만큼 `[PAD]` 토큰 추가

In [8]:
tokens = tokens + ['[PAD]'] + ['[PAD]']
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


어텐션 마스크 생성 : 모든 위치에서 어텐션 마스크값을 1로 설정하고 `[PAD] ` 토큰이 있는 위치에만 0을 설정

In [9]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


모든 토큰을 토큰 ID로 변환

In [10]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1045, 2293, 3000, 102, 0, 0]


token_ids와 attention_mask를 텐서로 변환

In [11]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

### 임베딩 추출하기

In [12]:
hidden_rep, cls_head = model(token_ids, attention_mask = attention_mask)

In [13]:
print(hidden_rep)

last_hidden_state


In [14]:
print(cls_head)

pooler_output


📌 모델이 반환하는 출력이 책의 실습 코드 [3.03. Generating BERT embedding .ipynb](https://github.com/PacktPublishing/Getting-Started-with-Google-BERT/blob/main/Chapter03/3.03.%20Generating%20BERT%20embedding%20.ipynb) 와 상이하다.

In [15]:
model(token_ids, attention_mask = attention_mask)

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0719,  0.2163,  0.0047,  ..., -0.5865,  0.2262,  0.1981],
                                                        [ 0.2236,  0.6536, -0.2294,  ..., -0.3547,  0.5517, -0.2367],
                                                        [ 1.0410,  0.7755,  1.0335,  ..., -0.5621,  0.5218, -0.0852],
                                                        ...,
                                                        [ 0.6156,  0.1036, -0.1875,  ..., -0.3799, -0.7008, -0.3500],
                                                        [ 0.0791,  0.4287,  0.4147,  ..., -0.2417,  0.2403,  0.0378],
                                                        [-0.0165,  0.2459,  0.4566,  ..., -0.2179,  0.1876,  0.0228]]],
                                                      grad_fn=<NativeLayerNormBackward0>)),
                                              ('pooler_output',
     

In [16]:
output = model(token_ids, attention_mask = attention_mask)

Cf. 책에서는 `last_hidden_state`를 `hidden_rep`로, `pooler_output`을 `cls_head`라고 기술했다.

In [17]:
last_hidden_state = output[0]
pooler_output = output[1]

`last_hidden_state`는 최종 인코더(12번째 인코더)에서만 얻은, 입력에 대한 모든 토큰의 임베딩(표현)을 포함

In [18]:
print(last_hidden_state.shape) # [batch_size, seqeunce_length, hidden_size]

torch.Size([1, 7, 768])


- last_hidden_state[0][0] : 첫 번째 토큰인 `[CLS]`의 표현 벡터
- last_hidden_state[0][1] : 두 번째 토큰인 `'i'`의 표현 벡터
- last_hidden_state[0][2] : 세 번째 토큰인 `'love'`의 표현 벡터

`pooler_output`은 `[CLS]` 토큰의 표현을 나타내며, 선형 및 tanh 활성화 함수에 의해 계산

In [19]:
print(pooler_output.shape) # [batch_size, hidden_size]

torch.Size([1, 768])


- `[CLS]` 토큰의 표현은 문장 전체의 표현 보유하므로, 'I love Paris' 문장의 표현으로 `pooler_output` 사용 가능